# Hartree Fock 

FC = eSC

F = T + V_NE + V_EE

S = overlap matrix

H2 and then H2O

T = Kinetic energy matrix

In [23]:
import sys
import numpy as np
from typing import List
from scipy import special

We consider H_2 molecule first
There is a 1s orbital per hydrogen atom
Each of them is given by a linear combination of basis functions, here Gaussians.

In [24]:
class primitive_gaussian():

    def __init__(self, 
                 alpha: float,
                 coeff: float,
                 coordinates: List,
                 l1: int = 0,
                 l2: int = 0,
                 l3: int = 0,
                 ) -> None:
        """ Constructor function for the primitive_gaussian class.

        l1, l2, l3 are equal to 0 for an s orbital. (focus on H_2 molecule first).
        """

        self.alpha = alpha
        self.coeff = coeff
        self.coordinates = np.array(coordinates)
        self.l1, self.l2, self.l3 = l1, l2, l3
        self.A = (2.0 * alpha / np.pi)**0.75  # normalization factor, + other temrs for l1, l2, l3 > 0

In [12]:
def overlap(molecule: List) -> None:
    nbasis = len(molecule)

    S = np.zeros([nbasis, nbasis])

    for i in range(nbasis):
        for j in range(nbasis):
            nprimitives_i = len(molecule[i])
            nprimitives_j = len(molecule[j])

            for k in range(nprimitives_i):
                for l in range(nprimitives_j):
                    
                    # normalization constant of the total overlap
                    N = molecule[i][k].A * molecule[j][l].A
                    p = molecule[i][k].alpha + molecule[j][l].alpha
                    q = molecule[i][k].alpha * molecule[j][l].alpha / p
                    Q = molecule[i][k].coordinates - molecule[j][l].coordinates
                    Q2 = np.dot(Q, Q)
                    
                    # this expression will be more complicated with non trivial l quantum number.
                    S[i, j] += N * molecule[i][k].coeff * molecule[j][l].coeff * np.exp(-q * Q2) * (np.pi / p) ** (3/2) 

    return S


In [13]:
def kinetic(molecule: List) -> None:
    nbasis = len(molecule)

    T = np.zeros([nbasis, nbasis])

    for i in range(nbasis):
        for j in range(nbasis):
            nprimitives_i = len(molecule[i])
            nprimitives_j = len(molecule[j])

            for k in range(nprimitives_i):
                for l in range(nprimitives_j):
                    c1c2 = molecule[i][k].coeff * molecule[j][l].coeff

                    # normalization constant of the total overlap
                    N = molecule[i][k].A * molecule[j][l].A
                    p = molecule[i][k].alpha + molecule[j][l].alpha
                    q = molecule[i][k].alpha * molecule[j][l].alpha / p
                    Q = molecule[i][k].coordinates - molecule[j][l].coordinates
                    Q2 = np.dot(Q, Q)
                    
                    P = molecule[i][k].alpha * molecule[i][k].coordinates + \
                        molecule[j][l].alpha * molecule[j][l].coordinates
                    Pp = P / p
                    PG = Pp - molecule[j][l].coordinates
                    PGx2 = PG[0] * PG[0]
                    PGy2 = PG[1] * PG[1]
                    PGz2 = PG[2] * PG[2]

                    s = N * c1c2 * np.exp(-q * Q2) * (np.pi / p) ** (3/2)

                    # this expression will be more complicated with non trivial l quantum number.
                    T[i, j] += 3 * molecule[j][l].alpha * s 
                    T[i, j] -= 2 * molecule[j][l].alpha * molecule[j][l].alpha * s * (PGx2 + 0.5/p)
                    T[i, j] -= 2 * molecule[j][l].alpha * molecule[j][l].alpha * s * (PGy2 + 0.5/p) 
                    T[i, j] -= 2 * molecule[j][l].alpha * molecule[j][l].alpha * s * (PGz2 + 0.5/p) 

    return T


In [ ]:
def boys(x, n):
    if x == 0:
        return 1.0 / (2 * n + 1)
    else:
        return special.gammainc(n + 0.5, x) * special.gamma(n + 0.5) * (1.0 / (2 * x**(n + 0.5)))

def electron_nuclear_attraction(molecule, atom_coordinates, Z):
    natoms = len(Z)
    nbasis = len(molecule)

    V_ne = np.zeros([nbasis, nbasis])

    for atom in range(natoms):
        for i in range(nbasis):
            for j in range(nbasis):
                nprimitives_i = len(molecule[i])
                nprimitives_j = len(molecule[j])

                for k in range(nprimitives_i):
                    for l in range(nprimitives_j):
                        c1c2 = molecule[i][k].coeff * molecule[j][l].coeff

                        # normalization constant of the total overlap
                        N = molecule[i][k].A * molecule[j][l].A
                        p = molecule[i][k].alpha + molecule[j][l].alpha
                        q = molecule[i][k].alpha * molecule[j][l].alpha / p
                        Q = molecule[i][k].coordinates - molecule[j][l].coordinates
                        Q2 = np.dot(Q, Q)
                        
                        P = molecule[i][k].alpha * molecule[i][k].coordinates + \
                            molecule[j][l].alpha * molecule[j][l].coordinates
                        Pp = P / p
                        PG = Pp - atom_coordinates[atom]
                        PG2 = np.dot(PG, PG)

                        # this expression will be more complicated with non trivial l quantum number.
                        V_ne[i, j] += -Z[atom] * N * c1c2 * np.exp(-q * Q2) * (2.0 * np.pi / p) * boys(p*PG2, 0)

    return V_ne
   

In [18]:
# STO-3G basis for 1s orbital on hydrogen
H1_pg1a = primitive_gaussian(alpha=0.3425250914E+01, 
                             coeff=0.1543289673E+00, 
                             coordinates=[0, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H1_pg1b = primitive_gaussian(alpha=0.6239137298E+00, 
                             coeff=0.5353281423E+00, 
                             coordinates=[0, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H1_pg1c = primitive_gaussian(alpha=0.1688554040E+00, 
                             coeff=0.4446345422E+00, 
                             coordinates=[0, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)

H2_pg1a = primitive_gaussian(alpha=0.3425250914E+01, 
                             coeff=0.1543289673E+00, 
                             coordinates=[1.4, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H2_pg1b = primitive_gaussian(alpha=0.6239137298E+00, 
                             coeff=0.5353281423E+00, 
                             coordinates=[1.4, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H2_pg1c = primitive_gaussian(alpha=0.1688554040E+00, 
                             coeff=0.4446345422E+00, 
                             coordinates=[1.4, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H1_1s = [H1_pg1a, H1_pg1b, H1_pg1c] # 1s orbital on the first hydrogen atom
H2_1s = [H2_pg1a, H2_pg1b, H2_pg1c] # 1s orbital on the second hydrogen atom
molecule = [H1_1s, H2_1s]
print("\nSTO-3G basis for H2:\n")
print("Overlap Matrix:\n", overlap(molecule))
print("\nKinetic Energy Matrix:\n", kinetic(molecule))


STO-3G basis for H2:

Overlap Matrix:
 [[1.         0.65931821]
 [0.65931821 1.        ]]

Kinetic Energy Matrix:
 [[0.76003188 0.23645466]
 [0.23645466 0.76003188]]


In [20]:
# 6-31G basis for 1s and 2s orbitals on hydrogen

H1_pg1a = primitive_gaussian(alpha=0.1873113696E+02, 
                             coeff=0.3349460434E-01, 
                             coordinates=[0, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H1_pg1b = primitive_gaussian(alpha=0.2825394365E+01, 
                             coeff=0.2347269535E+00, 
                             coordinates=[0, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H1_pg1c = primitive_gaussian(alpha=0.6401216923E+00, 
                             coeff=0.8137573261E+00, 
                             coordinates=[0, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H1_pg2a = primitive_gaussian(alpha=0.1612777588E+00, 
                             coeff=1.0000000, 
                             coordinates=[0, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)

H2_pg1a = primitive_gaussian(alpha=0.1873113696E+02, 
                             coeff=0.3349460434E-01, 
                             coordinates=[1.4, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)

H2_pg1b = primitive_gaussian(alpha=0.2825394365E+01, 
                             coeff=0.2347269535E+00, 
                             coordinates=[1.4, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H2_pg1c = primitive_gaussian(alpha=0.6401216923E+00, 
                             coeff=0.8137573261E+00, 
                             coordinates=[1.4, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H2_pg2a = primitive_gaussian(alpha=0.1612777588E+00, 
                             coeff=1.0000000, 
                             coordinates=[1.4, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)

H1_1s = [H1_pg1a, H1_pg1b, H1_pg1c] # 1s orbital on the first hydrogen atom
H1_2s = [H1_pg2a] # 2s orbital on the first hydrogen atom
H2_1s = [H2_pg1a, H2_pg1b, H2_pg1c] # 1s orbital on the second hydrogen atom
H2_2s = [H2_pg2a] # 2s orbital on the second hydrogen atom

molecule = [H1_1s, H1_2s, H2_1s, H2_2s]

print("\n6-31G basis for H2:\n")
print("Overlap Matrix:\n", overlap(molecule))
print("\nKinetic Energy Matrix:\n", kinetic(molecule))


6-31G basis for H2:

Overlap Matrix:
 [[1.         0.65829197 0.45453898 0.5087616 ]
 [0.65829197 1.         0.5087616  0.85380521]
 [0.45453898 0.5087616  1.         0.65829197]
 [0.5087616  0.85380521 0.65829197 1.        ]]

Kinetic Energy Matrix:
 [[1.39567838 0.259735   0.23200115 0.16601725]
 [0.259735   0.24191664 0.16601725 0.18478593]
 [0.23200115 0.16601725 1.39567838 0.259735  ]
 [0.16601725 0.18478593 0.259735   0.24191664]]
